# 1. Given optimal keffs, choose fewest keff changes--i.e., most sensitive ones
Two-step procedure:
1. Use favorite method to get optimal keffs
1. Run MILP to choose the fewest keffs differing from nominal keffs

$$
\begin{align*}
\min_{v,y,z} \quad & \sum_k \| (c^k)^T v^k - \mu^{*k} \| \\
\mathrm{s.t.} \quad & S^k v^k = 0 \\
& \sum_i x_i^k \leq c^k \\
& x_i^k = \alpha_i v_i^k + \beta_i z_i^k - \alpha_i z_i^k \\
& l_i^k \cdot y_i^k \leq z_i^k \leq u_i^k \cdot y_i^k \\
& l_i^k \leq v_i^k \leq u_i^k \\
& \sum_i y_i \leq K \\
& [\text{Duality gap constraint}] \\
& y_i \in \{0,1\}
\end{align*}
$$

For L1 norm:
$$
\begin{align*}
\min_{v,y,z} \quad & \sum_k sp^k + sn^k \\
\mathrm{s.t.} \quad & S^k v^k = 0 \\
& sp - sn = (c^k)^T v^k - \mu^{*k} \\
& \sum_i x_i^k \leq c^k \\
& x_i^k = \alpha_i v_i^k + \beta_i z_i^k - \alpha_i z_i^k \\
& l_i^k \cdot y_i^k \leq z_i^k \leq u_i^k \cdot y_i^k \\
& l_i^k \leq v_i^k \leq u_i^k \\
& \sum_i y_i \leq K \\
& [\text{Duality gap constraint}] \\
& y_i \in \{0,1\} \\
& sp, sn \geq 0
\end{align*}
$$

Alternatively,
$$
\begin{align*}
\min_{v,y,z} \quad & \sum_i y_i \leq K \\
\mathrm{s.t.} \quad & S^k v^k = 0 \\
& sp^k - sn^k = (c^k)^T v^k - \mu^{*k} \\
& \sum_i x_i^k \leq c^k \\
& x_i^k = \alpha_i v_i^k + \beta_i z_i^k - \alpha_i z_i^k \\
& l_i^k \cdot y_i^k \leq z_i^k \leq u_i^k \cdot y_i^k \\
& l_i^k \leq v_i^k \leq u_i^k \\
& sp^k + sn^k \leq f^k \\
& [\text{Duality gap constraint}] \\
& y_i \in \{0,1\} \\
& sp, sn \geq 0
\end{align*}
$$

With duality gap constraints:
$$
\begin{align*}
\min_{v,y,z} \quad & \sum_i y_i \leq K \\
\mathrm{s.t.} \quad & S^k v^k = 0 \\
& sp^k - sn^k = (c^k)^T v^k - \mu^{*k} \\
& \sum_i x_i^k \leq C^k \\
& x_i^k = \alpha_i v_i^k + \beta_i z_i^k - \alpha_i z_i^k \\
& l_i^k \cdot y_i^k \leq z_i^k \leq u_i^k \cdot y_i^k \\
& l_i^k \leq v_i^k \leq u_i^k \\
& sp^k + sn^k \leq f^k \\
& (c^k)^T v^k = w^a C + w^u u - w^l l \\
& w^S S + w^u - w^l + w^a a = (c^k)^T \\
& y_i \in \{0,1\} \\
& sp, sn \geq 0 \\
& w^u, w^l, w^a \geq 0 \\
& v^k, w^s, z \in \mathbb{R}
\end{align*}
$$

## MILP tips
- Callbacks via [Python API of Gurobi](http://www.gurobi.com/documentation/7.5/refman/py_python_api_overview.html#sec:Python)
    - Lazy constraints (e.g., for Benders decomposition): [Model.cbLazy](http://www.gurobi.com/documentation/7.5/refman/py_model_cblazy.html#pythonmethod:Model.cbLazy)
    - User cuts: [Model.cbCut](http://www.gurobi.com/documentation/7.5/refman/py_model_cbcut.html#pythonmethod:Model.cbCut)
- Eliminate incumbents via tighter McCormick relaxation of bilinear constraints
    - 20% speedup of radix method

Alternatively, run one-shot MINLP
- could reformulate to gigantic MILP, e.g., via radix reformulation

In [32]:
from cobra.io import load_json_model

ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')

## Simple one condition problem

In [112]:
from cobra import Metabolite, Reaction

class Constraint(Metabolite):
    pass

class Variable(Reaction):
    pass

In [123]:
F_ERROR = 0.05

### Create basic MILP using cobrapy
#ijomilp    = load_json_model('/home/laurence/ME/models/BiGG_M/json/iJO1366.json')
ijomilp = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')

crowding0 = ijomc.metabolites.crowding
cons_crowd = Constraint('crowding')
cons_crowd._constraint_sense = 'L'
cons_crowd._bound = crowding0._bound
xs = []
ys = []
zs = []
for rxn0 in crowding0.reactions:
    ### Add sum_i xi <= c
    x = Variable('x_'+rxn0.id)
    xs.append(x)
    x.add_metabolites({cons_crowd:1})
    ### Add binary constraints
    ### xki = αi vki + βi zki − αi zki
    ### --> xki - ai vki - bi zki + ai zki = 0
    cons_x = Constraint('cons_'+x.id)
    x.add_metabolites({cons_x:1.})
    ai = rxn0.metabolites[crowding0]
    rxn_fit = ijofit.reactions.get_by_id(rxn0.id)
    crowd_fit = ijofit.metabolites.crowding
    bi = rxn_fit.metabolites[crowd_fit]
    vi = ijomilp.reactions.get_by_id(rxn0.id)
    vi.add_metabolites({cons_x:-ai})
    zi = Variable('z_'+rxn0.id)
    zs.append(zi)
    zi.add_metabolites({cons_x:-bi + ai})
    ### lki⋅yki ≤ zki ≤ uki⋅yki
    yi = Variable('y_'+rxn0.id)
    ys.append(yi)
    yi.variable_kind = 'integer'
    yi.lower_bound = 0.
    yi.upper_bound = 1.
    cons_yl = Constraint('cons_binary_L_'+rxn0.id)
    cons_yl._bound = 0.
    cons_yl._constraint_sense = 'L'
    yi.add_metabolites({cons_yl:rxn_fit.lower_bound})
    zi.add_metabolites({cons_yl:-1.})
    cons_yg = Constraint('cons_binary_G_'+rxn0.id)
    cons_yg._bound = 0.
    cons_yg._constraint_sense = 'L'
    yi.add_metabolites({cons_yg:-rxn_fit.upper_bound})
    zi.add_metabolites({cons_yg:1})
    
ijomilp.add_reactions(xs)
ijomilp.add_reactions(ys)
ijomilp.add_reactions(zs)

### L1 norm objective 
for rxn in ijomilp.reactions:
    rxn.objective_coefficient = 0.

for y in ys:
    y.objective_coefficient = 1.
    
mu_p = Variable('mu_p')
mu_n = Variable('mu_n')
ijomilp.add_reactions([mu_p, mu_n])

mu_p.lower_bound = 0.
mu_n.lower_bound = 0.
#mu_p.objective_coefficient = 1.
#mu_n.objective_coefficient = 1.
### sp−sn - c'vk = -μ∗
cons_norm = Constraint('cons_norm')
mu_exp = 0.740
cons_norm._bound = -mu_exp
mu_p.add_metabolites({cons_norm:1.})
mu_n.add_metabolites({cons_norm:-1.})
rxn_obj = ijomilp.reactions.BIOMASS_Ec_iJO1366_core_53p95M
rxn_obj.add_metabolites({cons_norm:-1.})

### sp + sn <= f
cons_obj = Constraint('cons_obj')
cons_obj._constraint_sense = 'L'
cons_obj._bound = F_ERROR*mu_exp   # 1% error allowed
mu_p.add_metabolites({cons_obj:1.})
mu_n.add_metabolites({cons_obj:1.})

### Duality gap constraints
wSs = []
wus = []
wls = []
wa  = []

### mu = wa*C + wu*u - wl*l
### mu - wa*C - wu*u + wl*l = 0
cons_gap = Constraint('cons_duality_gap')

rxn_obj.add_metabolites({cons_gap:1.})

wa = Variable('wa')
wa.lower_bound = 0.
ijomilp.add_reaction(wa)

wa.add_metabolites({cons_gap:-cons_crowd._bound})

for met in ijomilp.metabolites:
    if not isinstance(met, Constraint):
        wSi = Variable('wS_'+met.id)
        wSs.append(wSi)
    
ijomilp.add_reactions(wSs)

for rxn in ijomilp.reactions:
    if not isinstance(rxn, Variable):
        wui = Variable('wu_'+rxn.id)
        wli = Variable('wl_'+rxn.id)
        wui.lower_bound = 0.
        wli.lower_bound = 0.
        wus.append(wui)
        wls.append(wli)
        wui.add_metabolites({cons_gap:-rxn.upper_bound})
        wli.add_metabolites({cons_gap:rxn.lower_bound})
        ### wS*S + wu - wl + wa*a = c
        cons_dual = Constraint('cons_dual_'+rxn.id)
        cons_dual._constraint_sense = 'E' # since v can be neg
        rxn0 = ijomc.reactions.get_by_id(rxn.id)
        cons_dual._bound = rxn0.objective_coefficient
        for met,si in rxn.metabolites.iteritems():
            if not isinstance(met, Constraint):        
                wSi = ijomilp.reactions.get_by_id('wS_'+met.id)
                wSi.add_metabolites({cons_dual:si})        
        wui.add_metabolites({cons_dual:1.})
        wli.add_metabolites({cons_dual:-1.})
        # if this rxn participates in crowding:
        if rxn.metabolites.has_key(cons_crowd):
            ai = rxn.metabolites[cons_crowd]
            wa.add_metabolites({cons_dual:ai})
    
ijomilp.add_reactions(wus)
ijomilp.add_reactions(wls)

In [130]:
ijomilp.metabolites.cons_dual_BIOMASS_Ec_iJO1366_core_53p95M._bound

1.0

In [140]:
cons_errs = []
for rxn in cons_gap.reactions:
    cons_errs.append(rxn.x * rxn.metabolites[cons_gap])
sum(cons_errs)

3.469446951953614e-17

In [124]:
F_ERROR = 0.01
cons_obj._bound = F_ERROR*mu_exp   # 1% error allowed

In [125]:
%%time
ijomilp.optimize(solver='gurobi', objective_sense='minimize')

CPU times: user 610 ms, sys: 23.3 ms, total: 633 ms
Wall time: 541 ms


<Solution 0.00 at 0x7f7d9ced7b90>

In [126]:
ijomilp.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7325999999972055

In [117]:
sum([y.x for y in ys])

0.0

In [111]:
# ijomilp.metabolites.cons_duality_gap.reactions

In [21]:
from gurobipy import *

#GRB.Callback.callback()

<bound method CallbackClass.callback of <gurobipy.CallbackClass object at 0x7f7e08a58390>>